In [113]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from datetime import datetime
import plotly
plotly.tools.set_credentials_file(username='bzjin', api_key='jmaLQFWVMgugFAITZZBC')
import plotly.plotly as py
import plotly.graph_objs as go


In [118]:
# "offender_offenses_ccs.csv" has 414,617 rows x 17 columns
offenses = pd.read_csv("offender_offenses_ccs.csv")
first = offenses.iloc[0:100000,:]
first.head()

,DCNumber,Sequence,OffenseDate,DateAdjudicated,County,CaseNumber,OffenseTerm,ProbationTerm,ParoleTerm,adjudicationcharge_descr,qualifier_descr,adjudication_descr
0,69177,1,03/16/60 00:00:00,03/16/60 00:00:00,MIAMI-DADE,NaN,9999998,0,9999998,"2ND DEG.MURD,DANGEROUS ACT",UNKNOWN,ADJUDICATION NOT WITHHELD
1,000004,1,06/29/11 00:00:00,04/11/12 00:00:00,HOLMES,1100163.0,0,20000,0,POSSESS METHAMPHETAMINE,PRINCIPAL,ADJUDICATION NOT WITHHELD
2,000004,2,06/29/11 00:00:00,04/11/12 00:00:00,HOLMES,1100163.0,0,30000,0,POSSESS METHAMPHETAMINE,PRINCIPAL,ADJUDICATION NOT WITHHELD
3,000741,1,09/29/60 00:00:00,09/30/60 00:00:00,HIGHLANDS,2133.0,9999998,0,9999998,1ST DG MUR/PREMED. OR ATT.,PRINCIPAL,ADJUDICATION NOT WITHHELD
4,000741,2,11/20/64 00:00:00,05/25/65 00:00:00,ORANGE,98097.0,50000,0,9999809,AGG ASSLT-W/WPN NO INTENT TO K,PRINCIPAL,ADJUDICATION NOT WITHHELD


In [119]:
#Charges
chargeSeries = pd.Series(first["adjudicationcharge_descr"])
chargeArray = np.array(chargeSeries, dtype=pd.Series)

unique, counts = np.unique(chargeArray, return_counts=True)

d = {}
for x in unique:
    d[x] = counts[np.where(unique==x)][0]


In [132]:
# Offense classifications from http://www.dc.state.fl.us/OffenderSearch/offensecategory.aspx#ED
classifications = pd.read_csv("offense_classifications.csv", low_memory=False)
classifications.head()

,code,name,category
0,1350,3+ BATTERY,Assault and Battery
1,3840,ABUSE ELDER/DISABLD,Assault and Battery
2,3841,ABUSE ELDER/DISBLD-HARM,Assault and Battery
3,1318,AGG ASSLT-INTENT COMMIT FELONY,Assault and Battery
4,1317,AGG ASSLT-W/WPN NO INTENT TO K,Assault and Battery


In [133]:
#This groups the types of ALL offenses together. An array stores those offenses that have not been accounted for or labelled yet. 
class_dict = {}

cate_Series = pd.Series(classifications["category"])
cate_Array = np.array(cate_Series, dtype=pd.Series)
unique, counts = np.unique(cate_Array, return_counts=True)

names = np.array(classifications["name"], dtype=pd.Series)

for x in unique:
    class_dict[x] = 0

missing_count = []
for x in list(d.keys()):
    if x in names:
        index = np.where(names == x)[0][0]
        category = classifications["category"][index]
        class_dict[category] = class_dict[category] + d[x]
    else: 
        missing_count.append(x)
    
print("Missing count: ", len(missing_count))
print(missing_count)

Missing count:  0
[]


In [131]:
list_offenses = list(class_dict.keys())
list_counts = list(class_dict.values())

offenses_data=[go.Bar(x=list_offenses, y=list_counts)]
py.iplot(offenses_data, filename='basic-bar')